In [1]:
import torch
import numpy as np

# Requires grad

In [ ]:
# optimization을 위해 gradient를 계산하고 싶은 tensor에는 requires_grad=True를 설정해줘야 함.
x = torch.ones(5, requires_grad=True) 
print(x) 

tensor([1., 1., 1., 1., 1.], requires_grad=True)
